# ID2214/FID3214 Assignment 3 Group no. [enter]
### Project members: 
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [1]:
import numpy as np
import pandas as pd
import time
import math
from sklearn.tree import DecisionTreeClassifier

## Reused functions from Assignment 1

In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
#Column filter
def create_column_filter(df): 
    exempt = ['CLASS']
    col_filter = exempt.copy()
    for col in df.columns: 
        if col not in exempt: 
            if (not df[col].isnull().values.all()) and (df[col].nunique() > 1): 
                col_filter.append(col)
    filtered = df.copy()
    filtered = filtered.loc[:, col_filter]
    return filtered, col_filter

def apply_column_filter(df, column_filter):
    column_filter = [c for c in column_filter if c not in ['CLASS']]
    return df[column_filter]

#Binning
def create_bins(df, nobins=10, bintype='equal-width'):
    to_rtn = df.copy()
    bins = {}
    for col in to_rtn.columns:
        if col != "CLASS" and col != "ID" and df[col].dtype in ["float64", "float32", "int64", "int32"]:
            if bintype == "equal-width":
                to_rtn[col], binR = pd.cut(to_rtn[col],nobins,retbins=True,duplicates="drop",labels=False)
                bins[col] = binR    
            elif bintype == "equal-size":
                to_rtn[col], binR = pd.qcut(to_rtn[col],q=nobins,retbins=True,duplicates="drop",labels=False)
                bins[col] = binR
            to_rtn[col] = to_rtn[col].astype("category")
            to_rtn[col] = to_rtn[col].cat.set_categories([str(i) for i in to_rtn[col].cat.categories], rename = True)
            #Infinity edges
            bins[col][0] = -np.inf
            bins[col][-1] = np.inf
        else:
            to_rtn[col] = to_rtn[col].astype('category')
    return to_rtn, bins

def apply_bins(df, binning): 
    exempt = ["ID", "CLASS"]
    binned_df = df.copy()
    for col in binned_df.columns: 
        if col not in exempt: 
            if binned_df[col].dtype == "int64" or binned_df[col].dtype == "float64": 
                bins = binning[col]
                bin_length = len(bins) - 1
                labels = np.arange(0, bin_length, 1)
                binned_df[col] = pd.cut(binned_df[col], bins, labels=labels)
                binned_df[col] = binned_df[col].astype('category')
    return binned_df

#Imputation
def create_imputation(df):
    to_rtn = df.copy()
    imp_dict = {}
    for col in to_rtn.columns:
        if col not in ['CLASS', 'ID']:
            if to_rtn[col].dtypes == "int" or to_rtn[col].dtype == "float":
                val = to_rtn[col].mean()
                to_rtn[col] = to_rtn[col].fillna(value=val)
            else:
                #Get first value of the mode, alternatively np.random could be used
                val = to_rtn[col].mode()[0]
                to_rtn[col] = to_rtn[col].fillna(value=val)
            imp_dict[col] = val
    #to_rtn = to_rtn.fillna(value=imp_dict)
    return to_rtn, imp_dict

def apply_imputation(df, imputation):
    to_rtn = df.copy()
    #No check for columns type because only rights type are included in the imputation dictionary
    to_rtn = to_rtn.fillna(value=imputation)
    return to_rtn


#Normalisation
def create_normalization(df, normalizationtype='minmax'):
    to_rtn = df.copy()
    norm_dict = {}
    colu = to_rtn.columns
    print(colu)
    for col in colu:
        #Check if col is numeric
        if pd.api.types.is_numeric_dtype(to_rtn[col]) and col not in ['CLASS', 'ID']:
            if normalizationtype == 'minmax':
                max = to_rtn[col].max()
                min = to_rtn[col].min()
                norm_dict[col] = (normalizationtype, min, max)
                #Normalize using minamx
                to_rtn[col] = to_rtn[col].apply(lambda x: (x-min)/(max-min))
            if normalizationtype == 'zscore':
                mean = to_rtn[col].mean()
                std = to_rtn[col].std()
                norm_dict[col] = ('zscore', mean, std)
                #Normalizing using zscore
                to_rtn[col] = to_rtn[col].apply(lambda x: (x-mean)/std)
    return to_rtn, norm_dict


def apply_normalization(df, norm_dict):
  to_rtn = df.copy()
  for col in to_rtn.columns:
    if pd.api.types.is_numeric_dtype(to_rtn[col]) and col not in ['CLASS', 'ID']:
        min = norm_dict[col][1]
        mean = norm_dict[col][1]
        max = std = norm_dict[col][2]
        normtype = norm_dict[col][0]
        if normtype == 'minmax':
            val_list = []
            # Calculate the normalized value in the range [0,1]
            for x in to_rtn[col]:
                v = (x-min)/(max-min)
                if v > 1.0:
                    v = 1
                if v < 0.0:
                    v = 0
                val_list.append(v)
            to_rtn[col] = val_list
            #to_rtn[col] = to_rtn[col].apply(lambda x: (x-min)/(max-min)) #Have to consider the [0,1] range
        if normtype == 'zscore':
            #Normalize with zscore
            to_rtn[col] = to_rtn[col].apply(lambda x: (x-mean)/std)
  return to_rtn


#One-hot encoding 
def create_one_hot(df):
    copydf = df.copy()
    one_hot = {}
    
    # filter all columns except ID and CLASS
    cols = copydf.loc[:, ~df.columns.isin(['ID', 'CLASS'])]
    # filter only categorial data
    cols = cols.select_dtypes(include=["object", "category"])
    
    for col in cols:
        # convert to get categories
        if copydf[col].dtype == 'object':
            copydf[col] = copydf[col].astype('category')
            
        # convert categorial to binary (one-hot)    
        dummies = pd.get_dummies(copydf[col])

        # create binary column for each category
        one_hot[col] = {}
        for cat in dummies:
            one_hot[col][cat] = col + '-' + cat
            copydf[col + '-' + cat] = dummies[cat]
            # convert to float type (hint 4)
            copydf[col + '-' + cat] = copydf[col + '-' + cat].astype('float')
            
        # drop original columns     
        copydf.drop(col, axis=1, inplace=True)
    
    return copydf, one_hot

def apply_one_hot(df,one_hot):
    df_new = df.copy()
    for col in df.columns:
        if col in one_hot:    
            for i in one_hot[col]:
                name = col + "_" + str(i)
                new_col = df[col]==i
                new_col = pd.Series(new_col.astype("int"))
                df_new[name] = new_col
            df_new = df_new.drop(columns = col, axis = 1)
            
    return df_new



#AUC 
def auc(df, correctlabels):   
    cols = df.columns
    cor_list = [(c == cols) for c in correctlabels]
    #print(cor_list)
    correct_filter = pd.DataFrame(cor_list, columns=cols)

    # Calculate binary AUC for each class label
    # Treating the predicted probability of this class for each instance as a score
    AUC = 0
    for col in df.columns:     
        # Map from each score to an array with number of true positives and true negative
        class_score = {score: [0, 0] for score in df[col]} # [positive, negative]
        
        for i in range(len(df[col])):  
            # We find score of the true positives and then the ones of true negatives
            score = df[col][i] # probability of class col row n
            is_positive = correct_filter[col][i] == True
            class_score[score] = [class_score[score][0] + is_positive,
                                  class_score[score][1] + ~is_positive]
        #We create a single reversed list of pairs
        sort_score = sorted(class_score, reverse=True)
        sorted_list = np.array([class_score[score] for score in sort_score])
        
        class_auc = cov_tp = 0
        tp, fp = sorted_list[:, 0], sorted_list[:, 1] 
        tot_tp, tot_fp = sum(tp), sum(fp)
        
        #Evaluate the AUC considering the 3 different cases
        for i in range(len(tp)):   
            if fp[i] == 0:
                #Increase up the. y-axis
                cov_tp += tp[i]
            elif tp[i] == 0:
                #Have rectangle to calculate
                class_auc += (cov_tp/tot_tp)*(fp[i]/tot_fp)
            else: 
                class_auc += (cov_tp/tot_tp)*(fp[i]/tot_fp)+(tp[i]/tot_tp)*(fp[i]/tot_fp)/2
                cov_tp += tp[i]
                
        frequency = dict(pd.Series(correctlabels).value_counts(normalize=True))
        AUC += frequency[col]*class_auc
    return AUC

#Accuracy
def accuracy(df, correctlabels):
    labels = df.idxmax(axis=1)
    truelabels = (labels == correctlabels).sum()
    accuracy = truelabels/len(df)
    
    return accuracy

#Brier Score
def brier_score(df, correctlabels): 
    label_df = pd.get_dummies(correctlabels)
    brier_score = np.mean(np.sum((df - label_df)**2, axis=1))
    
    return brier_score


## 1. Define the class RandomForest

In [3]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>

class RandomForest(): 

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
         

# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier


    def fit(self, df, no_trees=100): 
    
        self.labels = sorted(df['CLASS'].unique())
        df_c = df.copy()
        df_c, self.column_filter = create_column_filter(df_c)
        df_c, self.imputation = create_imputation(df_c)
        df_c, self.one_hot = create_one_hot(df_c)
        #Get list of the categories of the "CLASS" column
        
        labels_Y = np.array(df_c['CLASS'])
        
        df_c = df_c.drop(['CLASS'], axis=1)
        features_X = df_c.to_numpy()
        
        row_numbers = np.array(range(len(df_c.index)))
                
        #Create the no_trees models
        self.model = []
        for i in range(no_trees):
            #Get the bootstrap sample
            bootstrap_sample = np.random.choice(row_numbers, replace=True, size=len(row_numbers))
            Y = labels_Y[bootstrap_sample]
            X = features_X[bootstrap_sample]
    
            #Create the decision tree
            dec_tree = DecisionTreeClassifier(max_features=int(math.log(len(df_c.columns),2)))
            dec_tree = dec_tree.fit(X, Y)
         
            self.model.append(dec_tree)


#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 

    def predict(self, df): 
        df_c = df.copy()
        df_c = df_c.drop(['CLASS'], axis=1)
        
        #apply transformations to data
        df_c = apply_column_filter(df_c, self.column_filter)
        df_c = apply_imputation(df_c, self.imputation)
        df_c = apply_one_hot(df_c, self.one_hot)
        
        #creating empty probability matrix
        rows, cols = len(df_c.index),len(self.labels)
        prob_matrix = np.zeros((rows, cols))

        #iterating over each decision tree classifier
        for t in self.model:
            prob = np.matrix(t.predict_proba(df_c))
            prob_matrix = prob_matrix + prob
        #dividing to get the average prediction of models
        prob_matrix = np.divide(prob_matrix, len(self.model))
        to_rtn = pd.DataFrame(prob_matrix, columns=self.labels)
        
        return to_rtn

In [4]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.12 s.
Testing time: 0.15 s.
Accuracy: 0.9144
AUC: 0.9901
Brier score: 0.1793


In [5]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0228


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [6]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 



class RandomForest(): 

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.class_map = None
         

    def fit(self, df, no_trees=100): 
    
        self.labels = sorted(df['CLASS'].unique())
        df_c = df.copy()
        df_c, self.column_filter = create_column_filter(df_c)
        df_c, self.imputation = create_imputation(df_c)
        df_c, self.one_hot = create_one_hot(df_c)
        #Get list of the categories of the "CLASS" column
        
        labels_Y = np.array(df_c['CLASS'])
        
        df_c = df_c.drop(['CLASS'], axis=1)
        features_X = df_c.to_numpy()
        
        row_numbers = np.array(range(len(df_c.index)))
        
        ########################################################################################################
        # Create mapping
        self.class_map = dict((i,j) for j,i in enumerate(self.labels))
        ########################################################################################################
        
        #Create mapping
        #self.class_map = {}
        #i = 0
        #for l in self.labels:
            #self.class_map[l] = i
            #i += 1
        #print(self.class_map)

        
        #Create the no_trees models
        self.model = []
        for i in range(no_trees):
            #Get the bootstrap sample
            bootstrap_sample = np.random.choice(row_numbers, replace=True, size=len(row_numbers))
            Y = labels_Y[bootstrap_sample]
            X = features_X[bootstrap_sample]
    
            #Create the decision tree
            dec_tree = DecisionTreeClassifier(max_features=int(np.log2(len(df_c.columns))))#int(math.log(len(df_c.columns),2)))
            dec_tree = dec_tree.fit(X, Y)
         
            self.model.append(dec_tree)

    def predict(self, df): 
        df_c = df.copy()
        df_c = df_c.drop(['CLASS'], axis=1)
        
        #apply transformations to data
        df_c = apply_column_filter(df_c, self.column_filter)
        df_c = apply_imputation(df_c, self.imputation)
        df_c = apply_one_hot(df_c, self.one_hot)
        
        #creating empty probability matrix
        rows, cols = len(df_c.index),len(self.labels)
        prob_matrix = np.zeros((rows, cols))

        ########################################################################################################
        #iterating over each decision tree classifier
        for t in self.model:
            prob = np.array(t.predict_proba(df_c))
            #finding the correct probability matrix index to update)
            for col in range(prob.shape[1]):
                class_index = self.class_map[t.classes_[col]]
                prob_matrix[:, class_index] = prob_matrix[:, class_index] + prob[:, col]
       ########################################################################################################     
            #for r in range(prob.shape[0]):
                #for col in range(prob.shape[1]): 
                    #class_index = self.class_map[str(t.classes_[col])]
                    #prob_matrix[r][class_index] = prob_matrix[r][class_index] + prob[r][col]
        

        #dividing to get the average prediction of models
        prob_matrix = np.divide(prob_matrix, len(self.model))
        to_rtn = pd.DataFrame(prob_matrix, columns=self.labels)
        
        return to_rtn
    
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to     

In [7]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.12 s.
Testing time: 0.16 s.
Accuracy: 0.9510
AUC: 0.9729
Brier score: 0.0997


## 2b. Estimate predictive performance using out-of-bag predictions

In [8]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector

class RandomForest(): 

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.class_map = None
        self.oob_acc = None
         

    def fit(self, df, no_trees=100): 
    
        self.labels = sorted(df['CLASS'].unique())
        df_c = df.copy()
        df_c, self.column_filter = create_column_filter(df_c)
        df_c, self.imputation = create_imputation(df_c)
        df_c, self.one_hot = create_one_hot(df_c)
        #Get list of the categories of the "CLASS" column
        
        labels_Y = np.array(df_c['CLASS'])
        label_df = df_c['CLASS']
        
        df_c = df_c.drop(['CLASS'], axis=1)
        features_X = df_c.to_numpy()
        
        row_numbers = np.array(range(len(df_c.index)))
        
        ########################################################################################################
        # Create mapping
        self.class_map = dict((i,j) for j,i in enumerate(self.labels))
        ########################################################################################################
        #Create mapping
        #self.class_map = {}
        #i = 0
        #for l in self.labels:
            #self.class_map[l] = i
            #i += 1

        #Out-0f-bag structures 
        agg_oob = np.zeros((len(df_c.index),len(self.labels)))
        oob_count = np.zeros(len(df_c.index))
                
        #Create the no_trees models
        self.model = []
        for i in range(no_trees):
            #Get the bootstrap sample
            bootstrap_sample = np.random.choice(row_numbers, replace=True, size=len(row_numbers))
            #print(row_numbers)
            #print("\n")
            #print(bootstrap_sample)
            oob = set(row_numbers) - set(bootstrap_sample)
            oob = list(oob)
           
            Y = labels_Y[bootstrap_sample]
            X = features_X[bootstrap_sample]
    
            #Create the decision tree
            dec_tree = DecisionTreeClassifier(max_features=int(math.log(len(df_c.columns),2)))
            dec_tree = dec_tree.fit(X, Y)

      
            for i in oob: 
                
                oob_row  = df_c.iloc[i]
                oob_row = oob_row[None,:]
               
                oob_pred = np.array(dec_tree.predict_proba(oob_row))
                oob_pred = oob_pred[0]
                #update count 
                oob_count[i] += 1 
                #update the probabilities
                #agg_oob[i] = agg_oob[i] + oob_pred[0]
    
                for p in range(len(oob_pred)): 
                    class_index = self.class_map[str(dec_tree.classes_[p])]
                    agg_oob[i][class_index] = agg_oob[i][class_index] + oob_pred[p]
            
            self.model.append(dec_tree)

  
        reps = (agg_oob.shape[1], 1)
        count_mat = np.tile(oob_count,reps)
        count_mat = count_mat.transpose()
        agg_oob = np.divide(agg_oob,count_mat)
      
        #calculate accuracy 
        oob_predictions = pd.DataFrame(agg_oob, columns=self.labels)

        """
        oob_predictions = np.argmax(agg_oob, axis=1)
       
        for i in range(oob_predictions.shape[0]): 
                label = self.labels[oob_predictions[i]
                oob_predictions[i] = label
        
        oob_predictions = [self.labels[oob_predictions[i]] for i in oob_predictions]
        """

        self.oob_acc = accuracy(oob_predictions, label_df)
        print(self.oob_acc)
        


    def predict(self, df): 
        df_c = df.copy()
        df_c = df_c.drop(['CLASS'], axis=1)
        
        #apply transformations to data
        df_c = apply_column_filter(df_c, self.column_filter)
        df_c = apply_imputation(df_c, self.imputation)
        df_c = apply_one_hot(df_c, self.one_hot)
        
        #creating empty probability matrix
        rows, cols = len(df_c.index),len(self.labels)
        prob_matrix = np.zeros((rows, cols))

        ########################################################################################################
        #iterating over each decision tree classifier
        for t in self.model:
            prob = np.array(t.predict_proba(df_c))
            #finding the correct probability matrix index to update)
            for col in range(prob.shape[1]):
                class_index = self.class_map[t.classes_[col]]
                prob_matrix[:, class_index] = prob_matrix[:, class_index] + prob[:, col]
       ########################################################################################################     
        

        #dividing to get the average prediction of models
        prob_matrix = np.divide(prob_matrix, len(self.model))
        to_rtn = pd.DataFrame(prob_matrix, columns=self.labels)
        
        return to_rtn

In [11]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

0.9532293986636972
Training time: 5.09 s.
OOB accuracy: 0.9532
Testing time: 0.15 s.
Accuracy: 0.9443
AUC: 0.9729
Brier score: 0.0993


In [10]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

0.955456570155902
Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.